In [1]:
import pandas as pd
import numpy as np
import time
import os

### OD Matrix Data

In [2]:
file_path = '/local/tarciso/data/scaled-od-matrix/scaled-matrices/2017_05_02_scaled_od_matrix.csv'

In [3]:
scaled_od_matrix = pd.read_csv(file_path)

In [4]:
len(scaled_od_matrix)

22586

In [5]:
scaled_od_matrix.head()

,o_boarding_id,o_route,o_boarding_datetime,o_stopPointId,route,from_stop_id,to_stop_id
0,193.0,684,2017-05-02 04:44:56,39368.0,684,39368.0,36203.0
1,225.0,684,2017-05-02 04:46:47,39369.0,684,39369.0,31053.0
2,260.0,542,2017-05-02 04:48:31,31253.0,542,31253.0,30012.0
3,261.0,650,2017-05-02 04:48:32,36294.0,650,36294.0,31050.0
4,293.0,542,2017-05-02 04:49:20,31254.0,542,31254.0,31243.0


In [6]:
original_od_matrix = pd.read_csv('/local/tarciso/data/od-matrix/full/2017_05_02_full_od_trips.csv', parse_dates=['start_time', 'end_time'])

In [7]:
original_od_matrix.head()

,cardNum,user_trip_id,itinerary_id,leg_id,route,busCode,tripNum,from_stop_id,start_time,from_stop_lat,from_stop_lon,to_stop_id,end_time,to_stop_lat,to_stop_lon,leg_duration
0,2088629.0,271297,3,1,625,GA162,10.0,35713.0,2017-05-02 16:43:11,-25.496938,-49.290224,27639.0,2017-05-02 16:47:07,-25.492111,-49.293211,0 days 00:03:56.000000000
1,2088629.0,271297,3,3,607,HL328,5.0,26247.0,2017-05-02 16:54:03,-25.492115,-49.293043,25920.0,2017-05-02 17:12:07,-25.440314,-49.271795,0 days 00:18:04.000000000
2,3811772.0,271391,6,2,602,GR027,8.0,25402.0,2017-05-02 16:16:10,-25.481021,-49.292618,25478.0,2017-05-02 17:09:17,-25.489676,-49.243255,0 days 00:53:07.000000000
3,2376410.0,271392,1,1,535,EA077,14.0,36874.0,2017-05-02 16:44:09,-25.553459,-49.250936,30030.0,2017-05-02 17:04:48,-25.517281,-49.230276,0 days 00:20:39.000000000
4,2376410.0,271392,1,3,505,BL320,9.0,30024.0,2017-05-02 17:21:56,-25.516362,-49.230703,26264.0,2017-05-02 17:31:09,-25.481215,-49.246966,0 days 00:09:13.000000000


In [8]:
od_matrix_clean = original_od_matrix.filter(['cardNum','user_trip_id','start_time','route','from_stop_id','to_stop_id','from_stop_lat','from_stop_lon','to_stop_lat','to_stop_lon']) \
                                    .rename(index=str, columns={'user_trip_id':'boarding_id','start_time':'boarding_datetime'})

In [9]:
od_matrix_clean.head()

,cardNum,boarding_id,boarding_datetime,route,from_stop_id,to_stop_id,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,2088629.0,271297,2017-05-02 16:43:11,625,35713.0,27639.0,-25.496938,-49.290224,-25.492111,-49.293211
1,2088629.0,271297,2017-05-02 16:54:03,607,26247.0,25920.0,-25.492115,-49.293043,-25.440314,-49.271795
2,3811772.0,271391,2017-05-02 16:16:10,602,25402.0,25478.0,-25.481021,-49.292618,-25.489676,-49.243255
3,2376410.0,271392,2017-05-02 16:44:09,535,36874.0,30030.0,-25.553459,-49.250936,-25.517281,-49.230276
4,2376410.0,271392,2017-05-02 17:21:56,505,30024.0,26264.0,-25.516362,-49.230703,-25.481215,-49.246966


### User Trips Data

In [10]:
user_trips_data = pd.read_csv('/local/tarciso/data/enhanced-buste/user_trips/2017_05_02_user_trips.csv', parse_dates=['o_boarding_datetime','o_gps_datetime','next_o_boarding_datetime','next_o_gps_datetime'])

In [11]:
len(user_trips_data)

208965

In [12]:
user_trips_data.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_gps_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_gps_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
0,2529052.0,176.0,2017-05-02 04:43:01,2017-05-02 04:42:12,548,GA205,1.0,31594.0,-25.556861,-49.261999,...,2017-05-02 16:04:24,2017-05-02 16:03:34,545,KA689,6.0,33874.0,-25.528551,-49.328437,0 days 11:21:23.000000000,7.37699
1,2529052.0,253746.0,2017-05-02 16:04:24,2017-05-02 16:03:34,545,KA689,6.0,33874.0,-25.528551,-49.328437,...,2017-05-02 18:23:31,NaT,000,09018,NaN,26110.0,-25.487810,-49.294333,0 days 02:19:07.000000000,5.66785
2,2529052.0,351450.0,2017-05-02 18:23:34,NaT,000,09018,NaN,26110.0,-25.487810,-49.294333,...,2017-05-02 04:43:01,2017-05-02 04:42:12,548,GA205,1.0,31594.0,-25.556861,-49.261999,0 days 13:40:33.000000000,8.31146
3,2335000.0,191.0,2017-05-02 04:44:49,2017-05-02 04:43:48,684,HA602,1.0,39368.0,-25.588450,-49.341190,...,2017-05-02 15:19:06,2017-05-02 15:19:02,685,HA616,8.0,33864.0,-25.519727,-49.328470,0 days 10:34:17.000000000,7.71975
4,2335000.0,238340.0,2017-05-02 15:19:06,2017-05-02 15:19:02,685,HA616,8.0,33864.0,-25.519727,-49.328470,...,2017-05-02 04:44:49,2017-05-02 04:43:48,684,HA602,1.0,39368.0,-25.588450,-49.341190,0 days 10:34:17.000000000,7.71975


In [13]:
user_boardings = user_trips_data.filter(['cardNum','o_boarding_id'])

In [14]:
user_boardings.head()

,cardNum,o_boarding_id
0,2529052.0,176.0
1,2529052.0,253746.0
2,2529052.0,351450.0
3,2335000.0,191.0
4,2335000.0,238340.0


In [15]:
def get_router_id(query_date):
    INTERMEDIATE_OTP_DATE = pd.to_datetime("2017-06-30", format="%Y-%m-%d")
    router_id = ''

    if (query_date <= INTERMEDIATE_OTP_DATE):
        return 'ctba-2017-1'
    else:
        return 'ctba-2017-2'

In [16]:
#Adding stops location coordinates and parent station
gtfs_base_folderpath = '/local/tarciso/data/gtfs/'
file_date = pd.to_datetime(file_path.split('/')[-1].replace('_scaled_od_matrix.csv',''), format="%Y_%m_%d")
router_id = get_router_id(file_date)
stops_df = pd.read_csv(gtfs_base_folderpath + os.sep + router_id + os.sep + 'stops.txt')
stops_metadata = stops_df[['stop_id','stop_lat','stop_lon','parent_station']].rename(index=str,columns={'stop_id':'stopPointId'}) \
                    .drop(['parent_station'], axis=1)


In [17]:
stops_metadata.head()

,stopPointId,stop_lat,stop_lon
0,70,-25.435724,-49.306998
1,270,-25.501341,-49.237597
2,276,-25.451550,-49.214917
3,299,-25.451665,-49.215086
4,308,-25.501311,-49.237825


In [18]:
expanded_scaled_od = scaled_od_matrix.merge(user_boardings, on=['o_boarding_id']) \
                                        .rename(index=str, columns={'o_boarding_id':'boarding_id',
                                                                   'o_boarding_datetime':'boarding_datetime'}) \
                                        .filter(['cardNum','boarding_id','boarding_datetime','route','from_stop_id','to_stop_id']) \
                                        .merge(stops_metadata, left_on='from_stop_id', right_on='stopPointId') \
                                        .rename(index=str, columns={'stop_lat':'from_stop_lat', 'stop_lon':'from_stop_lon'}) \
                                        .drop('stopPointId', axis=1) \
                                        .merge(stops_metadata, left_on='to_stop_id', right_on='stopPointId') \
                                        .rename(index=str, columns={'stop_lat':'to_stop_lat', 'stop_lon':'to_stop_lon'}) \
                                        .drop('stopPointId', axis=1)

In [19]:
expanded_scaled_od.head()

,cardNum,boarding_id,boarding_datetime,route,from_stop_id,to_stop_id,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,715315.0,193.0,2017-05-02 04:44:56,684,39368.0,36203.0,-25.58845,-49.34119,-25.51516,-49.294444
1,2603551.0,2168.0,2017-05-02 05:16:29,684,39368.0,36203.0,-25.58845,-49.34119,-25.51516,-49.294444
2,10096791.0,92979.0,2017-05-02 07:52:38,684,39368.0,36203.0,-25.58845,-49.34119,-25.51516,-49.294444
3,836974.0,123256.0,2017-05-02 09:13:48,684,39368.0,36203.0,-25.58845,-49.34119,-25.51516,-49.294444
4,1314607.0,134103.0,2017-05-02 10:06:57,684,39368.0,36203.0,-25.58845,-49.34119,-25.51516,-49.294444


In [20]:
od_matrix_clean.head()

,cardNum,boarding_id,boarding_datetime,route,from_stop_id,to_stop_id,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,2088629.0,271297,2017-05-02 16:43:11,625,35713.0,27639.0,-25.496938,-49.290224,-25.492111,-49.293211
1,2088629.0,271297,2017-05-02 16:54:03,607,26247.0,25920.0,-25.492115,-49.293043,-25.440314,-49.271795
2,3811772.0,271391,2017-05-02 16:16:10,602,25402.0,25478.0,-25.481021,-49.292618,-25.489676,-49.243255
3,2376410.0,271392,2017-05-02 16:44:09,535,36874.0,30030.0,-25.553459,-49.250936,-25.517281,-49.230276
4,2376410.0,271392,2017-05-02 17:21:56,505,30024.0,26264.0,-25.516362,-49.230703,-25.481215,-49.246966


In [25]:
final_od_matrix = pd.concat([od_matrix_clean, expanded_scaled_od]) \
                        .sort_values(['cardNum','boarding_datetime']) \
                        .assign(boarding_datetime = lambda x: pd.to_datetime(x.boarding_datetime, format="%Y-%m-%d %H:%M:%S")) \
                        .rename(index=str, columns={'boarding_id':'trip_id'})

In [26]:
final_od_matrix.head(50)

,cardNum,trip_id,boarding_datetime,route,from_stop_id,to_stop_id,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
10968,229948.0,21697.0,2017-05-02 06:20:17,654,35350.0,27485.0,-25.492665,-49.303977,-25.492010,-49.293363
10969,229948.0,21697.0,2017-05-02 06:25:49,607,26247.0,26232.0,-25.492115,-49.293043,-25.407047,-49.252644
11168,229948.0,370349.0,2017-05-02 19:09:37,22,25683.0,26246.0,-25.410622,-49.248224,-25.491786,-49.293089
8911,230146.0,386340.0,2017-05-02 19:43:00,602,25798.0,25481.0,-25.525441,-49.230906,-25.495178,-49.240568
8974,230146.0,418505.0,2017-05-02 22:35:42,502,25423.0,27353.0,-25.496209,-49.240180,-25.516618,-49.230300
29052,233641.0,33642.0,2017-05-02 06:32:36,307,25725.0,25819.0,-25.423631,-49.202327,-25.428270,-49.284841
29053,233641.0,33642.0,2017-05-02 07:41:15,304,25819.0,26286.0,-25.428270,-49.284841,-25.441406,-49.347205
131364,257342.0,106029.0,2017-05-02 08:16:58,511,31195.0,33035.0,-25.498750,-49.247840,-25.481840,-49.249370
131365,257342.0,106029.0,2017-05-02 08:30:11,503,27474.0,27560.0,-25.481586,-49.247071,-25.433741,-49.270292
131470,257342.0,256226.0,2017-05-02 16:07:07,503,27560.0,26350.0,-25.433741,-49.270292,-25.481480,-49.247275


In [23]:
final_od_matrix.dtypes

cardNum                     float64
boarding_id                 float64
boarding_datetime    datetime64[ns]
route                         int64
from_stop_id                float64
to_stop_id                  float64
from_stop_lat               float64
from_stop_lon               float64
to_stop_lat                 float64
to_stop_lon                 float64
dtype: object

In [24]:
final_od_matrix.to_csv('/local/tarciso/data/scaled-od-matrix/final-odmat/' + file_date.strftime(format="%Y_%m_%d") + '_final_odmat.csv', index=False)

In [87]:
len(final_od_matrix)

245049

In [102]:
vis_dataset = final_od_matrix.rename(index=str, columns={'from_stop_id':'Name',
                                                        'from_stop_lat':'Latitude',
                                                        'from_stop_lon':'Longitude'}) \
                            .assign(Date = lambda x: x.boarding_datetime.dt.strftime('%m/%d/%Y'),
                                    Time = lambda x: x.boarding_datetime.dt.strftime('%H:%M:%S'),
                                    Load = 1) \
                            .filter(['Name','Latitude','Longitude','Date','Time','Load']) \
                            .sort_values(['Date','Time'])

In [103]:
vis_dataset.head()

,Name,Latitude,Longitude,Date,Time,Load
35426,31594.0,-25.556861,-49.261999,05/02/2017,04:42:12,1
35433,39368.0,-25.588450,-49.341190,05/02/2017,04:43:48,1
35428,39368.0,-25.588450,-49.341190,05/02/2017,04:43:48,1
35435,39368.0,-25.588450,-49.341190,05/02/2017,04:43:48,1
35430,31597.0,-25.560652,-49.256788,05/02/2017,04:44:39,1


In [104]:
sample_od_mat = vis_dataset.sample(20000)
sample_od_mat.to_csv('/local/tarciso/data/scaled-od-matrix/final-odmat/' + file_date.strftime(format="%Y_%m_%d") + '_final_odmat.csv', index=False)